# Background Tasks

> Background tasks are functions run after returning a response.

Useful for operations where you want the user to get a response quickly but you don't want or need the user waiting for it to finish. Typical scenarios include:

- User setup in complex systems where you can inform the user later in email that their account is complete
- Batch processes that take a significant amount of time (bulk email or API calls)
- Any other process where the user can be notified later by email, websocket, webhook, or pop-up

Background tasks are an easy-to-use wrapper over Python's concurrency mechanisms. They are used to improve user experience, often making apps feel faster to the end user.

## A simple background task example

In this example attaching a task to FtResponse by assigning it via the background argument. When the page is visited, it will display 'Simple Background Task Example' almost instantly, while in the terminal it will slowly count upward from 0.

``` {.python filename="main.py" code-line-numbers="true"}
from fasthtml.common import *
from starlette.background import BackgroundTask
from time import time, sleep

app, rt = fast_app()

def counter(loops:int): # <1>
    """Slowly print integers to the terminal"""
    for i in range(loops):
        print(i)
        sleep(i)

@rt
def index():
    task = BackgroundTask(counter, loops=5)  # <2>
    cts = Titled('Simple Background Task Example')
    return FtResponse(cts, background=task) # <3>

serve()
```

1. `counter` is our task function. There is nothing special about it, although it is a good practice for its arguments to be serializable as JSON
2. We use `starlette.background.BackgroundTask` to turn the counter function into a background task
3. `FtResponse` is called explicitly so we can attack the task to its background. Normally we don't need to call `FtResponse` explicitly, setting background tasks is the exception.

## A more complex example

In [3]:
from ContextKit import read_url

In [4]:
text = read_url('https://fastht.ml/docs/llms-ctx.txt')

In [5]:
%%ai
Read $(text) and create a FastHTML example of accessing a database to get 1000 users and then accessing an API for them 

Based on the context provided, I'll create a FastHTML example that demonstrates accessing a database to retrieve 1000 users and then accessing an API for each of them as a background task.

```python
from fasthtml.common import *
from starlette.background import BackgroundTask
import asyncio
import httpx
from datetime import datetime
import time

app, rt = fast_app()

db = database('data.db')

class User: email: str; account_updated:int
                
users = db.create(User, pk='email')
users


def process_all_users():
    """Process all users through the API"""
    start_time = time.time()

    for user in users():
        httpx.post(f'http://example.com/update-user/{user.email}')
        

@rt
def index():
    """Main page with form to trigger the background task"""
    return Titled("User API Processing", 
                 [
                     P("Click the button to retrieve 1000 users from the database and process them through an API."),
                     P("This operation will run in the background and you'll get an immediate response."),
                     Form(action="/process", method="post", 
                          contents=[
                              Submit("Start Processing")
                          ])
                 ])

@rt.post
def process():
    # Initialize the database if needed
    setup_db()
    
    # Get users from the database
    users = get_users(1000)
    
    # Create the background task
    task = BackgroundTask(process_all_users, users=users)
    
    # Return immediate response while processing continues in background
    return FtResponse(Titled("Processing Started", 
                            [
                                P(f"Started processing {len(users)} users in the background."),
                                P("You can check the server logs for progress updates."),
                                P("Results will be saved to api_results.txt when complete."),
                                A("Back to Home", href="/")
                            ]), 
                     background=task)

@rt
def results():
    """View the results if available"""
    try:
        with open('api_results.txt', 'r') as f:
            results = f.readlines()
        
        return Titled("API Processing Results",
                     [
                         P(f"Found {len(results)} processed users."),
                         Pre("\n".join(results[:20]) + "\n..." if len(results) > 20 else "\n".join(results)),
                         A("Back to Home", href="/")
                     ])
    except FileNotFoundError:
        return Titled("Results Not Available",
                     [
                         P("The API processing hasn't completed yet or hasn't been started."),
                         A("Back to Home", href="/")
                     ])

serve()
```

This example:

1. Creates a simulated SQLite database with 1000 users
2. Provides a simple web interface to trigger the background task
3. Uses `BackgroundTask` to process all users through a simulated API
4. Processes users in batches asynchronously to improve performance
5. Returns an immediate response to the user while processing continues
6. Logs progress to the terminal and saves results to a file

The key part is the `/process` route where we create the background task and attach it to the `FtResponse`. This allows the server to return a response immediately while continuing to process the API calls in the background.

::: {.callout-caution}
## Background tasks are not distributed task queues

While background tasks often provides the user with a faster experience, the server itself isn't accelerated. What that means is that processes are still happening, but are hidden. So if a server is struggling under the load of a lot of user activity, so long as it isn't an issue with HTTP, background tasks won't help with server load.

This is where full-fledged distributed task queue libraries like Celery and Dramatiq come into play. At the cost of dramatically increased complexity over background tasks they allow for the distribution of tasks over addition servers, as well as providing improved observability, retry mechanisms, and persistence in case of server shutdown.

In our experience, it's often better to build something with background tasks and then convert it to a task queue.
:::